In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
import gc
import pickle
import random
import datetime
import math

In [ ]:
# 把Google Drive挂载到Colab里
try:
    from google.colab import drive
    drive.mount('/content/drive')
except ImportError:
    pass

Mounted at /content/drive


In [ ]:
# 修改当前文件夹位置 假定notebook文件就在项目文件夹根目录
import os
def get_root_dir():
    if os.path.exists('/content/drive/MyDrive'):
        return '/content/drive/MyDrive/Colab/4-AMEX/AMEX Project/notebooks' #在Colab里
    else:
        return './' #在本地

#调用系统命令，相当于cd，但是直接!cd是不行的
os.chdir(get_root_dir())
get_root_dir()

'/content/drive/MyDrive/Colab/4-AMEX/AMEX Project/notebooks'

In [ ]:
part1_train = pd.read_parquet(path = "../data/7-FeatureSelection/Part1/train_interact_all.parquet", engine = 'pyarrow')
part2_train = pd.read_parquet(path = "../data/3-CompressedData-PartII/train_processed.parquet", engine = 'pyarrow')
part3_train = pd.read_parquet(path = "../data/5-LSTM-Embedding/train-LSTM-Embedding.parquet", engine = 'pyarrow')

In [ ]:
part1_train = part1_train.reset_index(drop=True)
part2_train = part2_train.reset_index(drop=True).drop(columns = ['customer_ID'])
part3_train = part3_train.reset_index(drop=True)

In [ ]:
part2_train.shape

(458913, 399)

In [ ]:
df = pd.concat([part1_train, part2_train, part3_train], axis=1)
del part1_train, part2_train, part3_train

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
 df.select_dtypes(include=['object'])

,customer_ID
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...
...,...
458908,ffff41c8a52833b56430603969b9ca48d208e7c192c6a4...
458909,ffff518bb2075e4816ee3fe9f3b152c57fc0e6f01bf7fd...
458910,ffff9984b999fccb2b6127635ed0736dda94e544e67e02...
458911,ffffa5c46bc8de74f5a4554e74e239c8dee6b9baf38814...


In [ ]:
to_be_scaled = df.drop(columns = ['B_30_last_0', 'B_30_last_1', 'B_30_last_2', 'B_38_last_1' ,'B_38_last_2',
 'B_38_last_3', 'B_38_last_4', 'B_38_last_5', 'B_38_last_6', 'B_38_last_7',
 'D_114_last_0', 'D_114_last_1' ,'D_117_last_0', 'D_117_last_2',
 'D_117_last_3', 'D_117_last_4', 'D_117_last_5', 'D_117_last_6',
 'D_117_last_7', 'D_120_last_0', 'D_120_last_1', 'D_64_last_0', 'D_64_last_2',
 'D_64_last_3', 'D_68_last_1', 'D_68_last_2', 'D_68_last_3' ,'D_68_last_4',
 'D_68_last_5', 'D_68_last_6','customer_ID'])

In [ ]:
ohe_columns = df[['B_30_last_0', 'B_30_last_1', 'B_30_last_2', 'B_38_last_1' ,'B_38_last_2',
 'B_38_last_3', 'B_38_last_4', 'B_38_last_5', 'B_38_last_6', 'B_38_last_7',
 'D_114_last_0', 'D_114_last_1' ,'D_117_last_0', 'D_117_last_2',
 'D_117_last_3', 'D_117_last_4', 'D_117_last_5', 'D_117_last_6',
 'D_117_last_7', 'D_120_last_0', 'D_120_last_1', 'D_64_last_0', 'D_64_last_2',
 'D_64_last_3', 'D_68_last_1', 'D_68_last_2', 'D_68_last_3' ,'D_68_last_4',
 'D_68_last_5', 'D_68_last_6']]

In [ ]:
scaler = StandardScaler()

In [ ]:
df.dtypes.value_counts()

float64    1538
float32     335
int8        113
int16        13
object        1
dtype: int64

In [ ]:
scaler.fit(to_be_scaled)
continuous_scaled = pd.DataFrame(scaler.transform(to_be_scaled), columns = to_be_scaled.columns)

In [ ]:
continuous_scaled

,B_38&5&S_9,D_114&1&S_13,D_114&0&D_79,D_64&-1&D_91,D_117&4&R_4,D_64&0&R_4,B_30&0&R_2,D_114&1&D_111,D_114&1&D_74,B_38&1&D_91,...,LSTM-Embedding91,LSTM-Embedding92,LSTM-Embedding93,LSTM-Embedding94,LSTM-Embedding95,LSTM-Embedding96,LSTM-Embedding97,LSTM-Embedding98,LSTM-Embedding99,LSTM-Embedding100
0,-0.219021,2.189610,-0.190002,0.309325,-0.140324,-0.179559,-0.270268,-0.943571,-0.269386,0.134435,...,-0.471321,-0.409648,-0.470557,-0.400081,-0.463026,-0.468410,-0.470535,0.791406,-0.470564,-0.470546
1,-0.219021,-0.024353,-0.190002,0.309325,-0.140324,-0.179559,-0.270268,-0.943571,-0.468583,0.134435,...,-0.471322,-0.409841,-0.470557,-0.400081,-0.463026,-0.468411,-0.470536,0.791408,-0.470565,-0.470547
2,-0.219021,-0.633330,-0.190002,0.309325,-0.140324,-0.179559,-0.270268,-0.461414,-0.468583,0.134435,...,-0.471321,-0.407935,-0.470557,-0.400081,-0.463026,-0.468411,-0.470536,0.791391,-0.470564,-0.470547
3,-0.219021,1.193602,-0.190002,0.309325,-0.140324,-0.179559,-0.270268,-0.943571,-0.368984,0.134435,...,-0.470763,-0.226719,-0.470418,-0.399964,-0.462560,-0.468273,-0.470397,0.789631,-0.470425,-0.470407
4,-0.219021,-0.633330,-0.190002,0.309325,-0.140324,-0.179559,-0.270268,-0.943571,0.328206,0.134435,...,-0.471322,-0.409891,-0.470557,-0.400081,-0.463026,-0.468411,-0.470536,0.791409,-0.470565,-0.470547
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458908,-0.219021,-0.633330,-0.190002,0.309325,-0.140324,-0.179559,-0.270268,1.145774,-0.468583,0.134435,...,-0.471320,-0.403443,-0.470556,-0.400080,-0.463022,-0.468410,-0.470535,0.791345,-0.470564,-0.470546
458909,-0.219021,-0.633330,-0.190002,0.309325,-0.140324,-0.179559,-0.270268,-0.943571,1.025397,0.134435,...,-0.471174,-0.398179,-0.470409,-0.399956,-0.462885,-0.468263,-0.470387,0.791162,-0.470416,-0.470398
458910,-0.219021,1.244000,-0.190002,0.309325,-0.140324,-0.179559,-0.270268,-0.943571,-0.468583,0.134435,...,-0.471321,-0.409695,-0.470557,-0.400081,-0.463026,-0.468411,-0.470536,0.791407,-0.470565,-0.470547
458911,-0.219021,-0.633330,-0.190002,0.309325,-0.140324,-0.179559,-0.270268,-0.461414,-0.070188,0.134435,...,1.100740,3.361083,1.113948,0.938041,1.050043,1.105941,1.115029,-0.192039,1.114796,1.113991


In [ ]:
df = pd.concat([pd.DataFrame(df['customer_ID']),continuous_scaled,ohe_columns], axis=1)

In [ ]:
df.to_parquet('../data/8-CombinedData/combined_data.parquet', index=False)

In [ ]:
train_labels= pd.read_csv("../data/1-original-data/train_labels.csv")

In [ ]:
df = df.merge(train_labels, on ='customer_ID')

In [ ]:
df.shape

(458913, 2001)

In [ ]:
df.head()

,customer_ID,B_38&5&S_9,D_114&1&S_13,D_114&0&D_79,D_64&-1&D_91,D_117&4&R_4,D_64&0&R_4,B_30&0&R_2,D_114&1&D_111,D_114&1&D_74,...,D_64_last_0,D_64_last_2,D_64_last_3,D_68_last_1,D_68_last_2,D_68_last_3,D_68_last_4,D_68_last_5,D_68_last_6,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,-0.219021,2.189610,-0.190002,0.309325,-0.140324,-0.179559,-0.270268,-0.943571,-0.269386,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,-0.219021,-0.024353,-0.190002,0.309325,-0.140324,-0.179559,-0.270268,-0.943571,-0.468583,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,-0.219021,-0.633330,-0.190002,0.309325,-0.140324,-0.179559,-0.270268,-0.461414,-0.468583,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,-0.219021,1.193602,-0.190002,0.309325,-0.140324,-0.179559,-0.270268,-0.943571,-0.368984,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,-0.219021,-0.633330,-0.190002,0.309325,-0.140324,-0.179559,-0.270268,-0.943571,0.328206,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0


In [ ]:
df.to_parquet('../data/8-CombinedData/combined-data-train.parquet', index=False)